In [1]:
import torch
import numpy as np
from torch import nn
torch.set_default_tensor_type('torch.FloatTensor')

In [2]:
num_inputs=2
num_examples=1000
true_w=[2,-3.4]
true_b=4.2
features=torch.tensor(np.random.normal(0,1,(num_examples,num_inputs)),dtype=torch.float)
labels=true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels+=torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float)

In [3]:
import torch.utils.data as Data
batch_size=10
dataset=Data.TensorDataset(features,labels)
data_iter=Data.DataLoader(dataset,batch_size,shuffle=True,num_workers=2)

In [4]:
for X,Y in data_iter:
    print(X,Y)
    break

tensor([[-0.6158,  0.0951],
        [ 1.8702,  0.2470],
        [-0.5196,  0.2866],
        [ 2.0951,  0.8574],
        [ 0.3864,  1.2216],
        [-0.7796,  1.3767],
        [ 0.4792, -0.0995],
        [ 1.3376, -0.9722],
        [-1.3298, -1.2402],
        [ 0.6083, -0.4989]]) tensor([ 2.6524,  7.1086,  2.1748,  5.4894,  0.8231, -2.0350,  5.5040, 10.1863,
         5.7654,  7.1190])


In [5]:
class LinearNet(nn.Module):
    def __init__(self,n_features):
        super(LinearNet,self).__init__()
        self.linear=nn.Linear(n_features,1)
    def forward(self,x):
        y=self.linear(x)
        return y

In [6]:
net=LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [7]:
#写法一
net=nn.Sequential(
    nn.Linear(num_inputs,1)
    #此处还可传入其它层
)

#写法二
net=nn.Sequential()
net.add_module('linear',nn.Linear(num_inputs,1))
#net.add_module......

#写法三
from collections import OrderedDict
net=nn.Sequential(OrderedDict([('linear',nn.Linear(num_inputs,1))
        #......                      
        ]))

In [8]:
print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [9]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.2140,  0.2817]], requires_grad=True)
Parameter containing:
tensor([0.2808], requires_grad=True)


In [10]:
from torch.nn import init

init.normal_(net[0].weight,mean=0.0,std=0.01)
init.constant_(net[0].bias,val=0.0)

Parameter containing:
tensor([0.], requires_grad=True)

In [11]:
loss=nn.MSELoss()

In [12]:
import torch.optim as optim

optimizer=optim.SGD(net.parameters(),lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [15]:
epochs=5
for epoch in range(epochs):
    for X,Y in data_iter:
        output=net(X)
        l=loss(output,Y.view(size=output.size()))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss:%f' % (epoch+1,l.item()))

epoch 1, loss:0.000165
epoch 2, loss:0.000148
epoch 3, loss:0.000099
epoch 4, loss:0.000098
epoch 5, loss:0.000086


In [16]:
dense=net[0]
print(true_w,dense.weight)
print(true_b,dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 2.0010, -3.3995]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1989], requires_grad=True)
